6.**[22 points] Spam classification**

(a)

In [31]:
import collections

import numpy as np

import src.util as util
import src.svm as svm


def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    return message.lower().split()
    # *** END CODE HERE ***


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message. 

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***

    # 自己写的太烂了
    # count = {}
    # is_added = {}
    # word2indices = {}
    # word2indices_size = 0
    # for message in messages:
    #     lst = get_words(message)
    #     for word in lst:
    #         if is_added.get(word):
    #             continue
    #         else:
    #             if count.get(word) is None:
    #                 count[word] = 1
    #             else:
    #                 count[word] += 1
    #                 if count[word] == 5:
    #                     is_added[word] = True
    #                     word2indices[word] = word2indices_size
    #                     word2indices_size += 1
    # print(word2indices)
    # return word2indices

    words = [word for message in messages for word in get_words(message)] # get words

    words_counts = collections.Counter(words) # use collections.Counter method to get a dictionary of word counts

    freq_words = [word for word,count in words_counts.items() if count >= 5]

    return {word:index for index, word in enumerate(freq_words)}

    # *** END CODE HERE ***


def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each 
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that 
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    m,n = len(messages),len(word_dictionary)
    matrix = np.zeros((m,n))
    words_counts= [collections.Counter(get_words(message)) for message in messages]
    for i in range(m):
        for word, count in words_counts[i].items():
            if word in word_dictionary:
                matrix[i][word_dictionary[word]] += count # This is += count, not += 1 !!!
    return matrix
    # *** END CODE HERE ***

(b) When inplement \verb|predict_from_naive_bayes_model|, we use the formula:
\begin{align*}
p(y=1|x) &= \frac{(\prod_{j=1}^n p(x_j|y=1)^x_j)p(y=1)}{(\prod_{j=1}^n p(x_j|y=1)^x_j)p(y=1)+(\prod_{j=1}^n p(x_j|y=0)^x_j)p(y=0)} \\
         &= \frac{1}{1+\frac{(\prod_{j=1}^n p(x_j|y=0)^{\# of x_j})p(y=0)}{(\prod_{j=1}^n p(x_j|y=1)^{\# of x_j})p(y=1)}} \\
         &= \frac{1}{1+\exp \left\{ \sum_{j=1}^n (\# of x_j) \log{ p(x_j|y=0)} - \sum_{j=1}^n (\# of x_j) \log{ p(x_j|y=1)}+\log{\frac{1-\phi}{\phi}} \right\} } 
\end{align*}
Then we can use $\sum_{j=1}^n (\# of x_j) \log{ p(x_j|y=0)} - \sum_{j=1}^n (\# of x_j) \log{ p(x_j|y=1)}+\log{\frac{1-\phi}{\phi}}$ as an indicator.

In [32]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    V_size = matrix.shape[1]
    phi_y = np.mean(labels)

    # phi_k_y1 = (np.sum(matrix[labels == 1],axis=0) + 1)/ (np.sum(np.sum(matrix,axis=1)[labels == 1]) + V_size)
    # phi_k_y0 = (np.sum(matrix[labels == 0],axis=0) + 1)/ (np.sum(np.sum(matrix,axis=1)[labels == 0]) + V_size)

    phi_k_y1 = (np.sum(matrix[labels == 1],axis=0) + 1)/ (np.sum(matrix[labels == 1]) + V_size)
    phi_k_y0 = (np.sum(matrix[labels == 0],axis=0) + 1)/ (np.sum(matrix[labels == 0]) + V_size)

    return (phi_k_y0,phi_k_y1,phi_y)
    # *** END CODE HERE ***


def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    return matrix @ (np.log(model[1]) - np.log(model[0]))+np.log(model[2]/(1-model[2])) >= 0
    # *** END CODE HERE ***

(c)

In [33]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***

    # These two inplement is the same:

    # rank = np.log(model[1]/model[0])
    # ids = np.flipud(np.argsort(rank))[0:5]
    
    rank = -(np.log(model[1])-np.log(model[0]))
    ids = np.argsort(rank)[0:5]

    print(ids)
    
    keys = list(dictionary.keys())
    return [keys[i] for i in ids]
    # *** END CODE HERE ***

(d)

In [34]:
def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider
    
    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***
    accuracy = [0]*len(radius_to_consider)
    for i,radius in enumerate(radius_to_consider):
        val_pred = svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, radius)
        accuracy[i] = np.mean(val_labels==val_pred)
    return radius_to_consider[np.argmax(accuracy)]
    # *** END CODE HERE ***

**Result**

In [35]:
def main():
    train_messages, train_labels = util.load_spam_dataset('./data/ds6_train.tsv')
    # print(train_messages)
    val_messages, val_labels = util.load_spam_dataset('./data/ds6_val.tsv')
    test_messages, test_labels = util.load_spam_dataset('./data/ds6_test.tsv')
    
    dictionary = create_dictionary(train_messages)
    # print(dictionary)

    util.write_json('./output/p06_dictionary', dictionary)

    train_matrix = transform_text(train_messages, dictionary)

    np.savetxt('./output/p06_sample_train_matrix', train_matrix[:100,:])

    val_matrix = transform_text(val_messages, dictionary)
    test_matrix = transform_text(test_messages, dictionary)

    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

    np.savetxt('./output/p06_naive_bayes_predictions', naive_bayes_predictions)

    naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

    print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

    top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

    print('The top 5 indicative words for Naive Bayes are: ', top_5_words)

    util.write_json('./output/p06_top_indicative_words', top_5_words)

    optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])

    util.write_json('./output/p06_optimal_radius', optimal_radius)

    print('The optimal SVM radius was {}'.format(optimal_radius))

    svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)

    svm_accuracy = np.mean(svm_predictions == test_labels)

    print('The SVM model had an accuracy of {} on the testing set'.format(svm_accuracy, optimal_radius))


if __name__ == "__main__":
    main()

Naive Bayes had an accuracy of 0.978494623655914 on the testing set
[310 571 991 450 567]
The top 5 indicative words for Naive Bayes are:  ['claim', 'won', 'prize', 'tone', 'urgent!']
The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9695340501792115 on the testing set


**Conclusion.** This problem is really challengable, the procedure is 
- First, read the messages in, then calculate the frequent words (about 1700 words) for each message, then get a numpy array with each row for a message
  and each column for a frequent word.
- Second, use *multinomial event model* and *Laplace smoothing* in sub-problem (b). 
  Note that the inplement of multinomial event model can be done by logarithms.
- Third, SVM is also a method. 